In [ ]:
#https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/
# NOTE, you need to install the apyori library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori

data = pd.read_csv('mental_health_CLEAN.csv')
data.head()

In [ ]:
records = []
for i in range(0,data.shape[0]):
    records.append([str(data.values[i,j]) for j in range(0,data.shape[1])])

association_rules = apriori(records, min_support = 0.0045, min_confidence = 0.2, min_lift=3, min_length = 2)
results = list(association_rules)

In [ ]:
print(len(association_rules))
print(association_rules[0])